In [1]:
import numpy as np
import itertools
np.seterr(divide='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
class Bar:
    def __init__(self, title, dia, length, number):
        self.title = title
        self.dia = dia
        self.length = length
        self.number = number

In [3]:
def get_coefficien_list(L, bar_length_list):
    bar_length_min = min(bar_length_list)
    max_iter = int(L/bar_length_min)+1
    co_efficient_list = np.arange(max_iter)

    product_arguments = [co_efficient_list]*len(bar_length_list)

    # print(product_arguments)
    return list(itertools.product(*product_arguments))

In [4]:
def get_coefficien_arr(L, bar_length_list):
    coefficien_arr = np.array(get_coefficien_list(L, bar_length_list))
    bar_length_arr = np.array(bar_length_list)

    sum_arr = np.sum(coefficien_arr*bar_length_arr, axis=1)
    # Remove all item which sum length = 0 or greater than L
    condition = (sum_arr == 0) | (sum_arr > L)
    condition = np.invert(condition)

    return coefficien_arr[condition]

In [5]:
def get_sorted_list(L,bar_length_list):
    coeff_arr = get_coefficien_arr(L,bar_length_list)
    westageArr =L-np.sum(coeff_arr * bar_length_list, axis=1)
    indexArr = np.argsort(westageArr)
    return coeff_arr[indexArr]

## Now Define the Bar Parameters

In [6]:
L = 12
bar_length_list = [2.5, 3, 4]

bar1 = Bar('A', 12, 1.5, 99)
bar2 = Bar('B', 12, 2, 128)
bar3 = Bar('C', 12, 4, 328)
bar4 = Bar('D', 12, 5.5, 854)
bar5 = Bar('E', 12, 7.5, 530)
bar6 = Bar('E', 12, 8.5, 745)

In [7]:
print(bar1.length)

1.5


In [8]:
barList = [bar1, bar2, bar3,bar4,bar5,bar6]

barLengthList = np.array([bar1.length, bar2.length, bar3.length,bar4.length,bar5.length,bar6.length])

**Now Find out the Combination Array**

In [9]:
bar_number_array = np.array([bar1.number,bar2.number,bar3.number,bar4.number,bar5.number,bar6.number])
bar_number_array

array([ 99, 128, 328, 854, 530, 745])

In [10]:
sorted_arr = get_sorted_list(L,barLengthList)
sorted_arr

array([[8, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 1],
       [0, 2, 2, 0, 0, 0],
       [0, 4, 1, 0, 0, 0],
       [3, 0, 0, 0, 1, 0],
       [3, 1, 0, 1, 0, 0],
       [0, 0, 3, 0, 0, 0],
       [0, 6, 0, 0, 0, 0],
       [4, 1, 1, 0, 0, 0],
       [4, 3, 0, 0, 0, 0],
       [0, 3, 0, 1, 0, 0],
       [5, 2, 0, 0, 0, 0],
       [1, 1, 2, 0, 0, 0],
       [1, 3, 1, 0, 0, 0],
       [4, 0, 0, 1, 0, 0],
       [1, 5, 0, 0, 0, 0],
       [0, 2, 0, 0, 1, 0],
       [0, 1, 1, 1, 0, 0],
       [5, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0],
       [2, 0, 0, 0, 0, 1],
       [2, 2, 1, 0, 0, 0],
       [2, 4, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0],
       [6, 1, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 0],
       [2, 0, 2, 0, 0, 0],
       [1, 2, 0, 1, 0, 0],
       [0, 0, 0, 2, 0, 0],
       [2, 1, 0, 1, 0, 0],
       [2, 0, 0, 0, 1, 0],
       [3, 1, 1, 0, 0, 0],
       [7, 0, 0, 0, 0, 0],
       [3, 3, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1],
       [4, 2, 0, 0, 0, 0],
       [0, 3, 1, 0, 0, 0],
 

In [11]:
def get_results(L,bar_number_array,bar_length_arr):
    results = []
    sorted_arr = get_sorted_list(L,bar_length_arr)
    for comb in sorted_arr:
        if bar_number_array.sum()==0:
            break
        cond = bar_number_array!=0
        if comb.sum()==(comb*cond).sum():
            min_arr = bar_number_array/comb
            min_arr[min_arr==np.nan] = np.inf
#             print(min_arr)
            cut = int(np.nanmin(min_arr))
            
            if cut !=0:
                bar_number_array = bar_number_array - comb*cut
                westage = L-(bar_length_arr*comb).sum()
                result = {'cut':cut,'combination':comb,'westage_per_cut':westage,'total_westage':cut*westage,'L':L}
                results.append(result)
    
    return results
    
    

In [12]:
results = get_results(L,bar_number_array,barLengthList)
results

C:\Users\sohel\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


[{'cut': 12,
  'combination': array([8, 0, 0, 0, 0, 0]),
  'westage_per_cut': 0.0,
  'total_westage': 0.0,
  'L': 12},
 {'cut': 3,
  'combination': array([1, 1, 0, 0, 0, 1]),
  'westage_per_cut': 0.0,
  'total_westage': 0.0,
  'L': 12},
 {'cut': 62,
  'combination': array([0, 2, 2, 0, 0, 0]),
  'westage_per_cut': 0.0,
  'total_westage': 0.0,
  'L': 12},
 {'cut': 68,
  'combination': array([0, 0, 3, 0, 0, 0]),
  'westage_per_cut': 0.0,
  'total_westage': 0.0,
  'L': 12},
 {'cut': 427,
  'combination': array([0, 0, 0, 2, 0, 0]),
  'westage_per_cut': 1.0,
  'total_westage': 427.0,
  'L': 12},
 {'cut': 1,
  'combination': array([0, 1, 0, 0, 0, 1]),
  'westage_per_cut': 1.5,
  'total_westage': 1.5,
  'L': 12},
 {'cut': 741,
  'combination': array([0, 0, 0, 0, 0, 1]),
  'westage_per_cut': 3.5,
  'total_westage': 2593.5,
  'L': 12},
 {'cut': 530,
  'combination': array([0, 0, 0, 0, 1, 0]),
  'westage_per_cut': 4.5,
  'total_westage': 2385.0,
  'L': 12}]

In [13]:
for x in results:
    print((np.array(x['combination'])*x['cut']))

[96  0  0  0  0  0]
[3 3 0 0 0 3]
[  0 124 124   0   0   0]
[  0   0 204   0   0   0]
[  0   0   0 854   0   0]
[0 1 0 0 0 1]
[  0   0   0   0   0 741]
[  0   0   0   0 530   0]


In [14]:
total_westage =0;
for x in results:
    total_westage= total_westage+x['total_westage']

In [15]:
total_westage

5407.0

In [16]:
optimized_list = []
distribution_list = []

for x in results:
    if x['westage_per_cut']==0 or x['westage_per_cut']<barLengthList.min():
        optimized_list.append(x)
    else:
        distribution_list.append(x)

In [17]:
optimized_list

[{'cut': 12,
  'combination': array([8, 0, 0, 0, 0, 0]),
  'westage_per_cut': 0.0,
  'total_westage': 0.0,
  'L': 12},
 {'cut': 3,
  'combination': array([1, 1, 0, 0, 0, 1]),
  'westage_per_cut': 0.0,
  'total_westage': 0.0,
  'L': 12},
 {'cut': 62,
  'combination': array([0, 2, 2, 0, 0, 0]),
  'westage_per_cut': 0.0,
  'total_westage': 0.0,
  'L': 12},
 {'cut': 68,
  'combination': array([0, 0, 3, 0, 0, 0]),
  'westage_per_cut': 0.0,
  'total_westage': 0.0,
  'L': 12},
 {'cut': 427,
  'combination': array([0, 0, 0, 2, 0, 0]),
  'westage_per_cut': 1.0,
  'total_westage': 427.0,
  'L': 12}]

In [18]:
distribution_list

[{'cut': 1,
  'combination': array([0, 1, 0, 0, 0, 1]),
  'westage_per_cut': 1.5,
  'total_westage': 1.5,
  'L': 12},
 {'cut': 741,
  'combination': array([0, 0, 0, 0, 0, 1]),
  'westage_per_cut': 3.5,
  'total_westage': 2593.5,
  'L': 12},
 {'cut': 530,
  'combination': array([0, 0, 0, 0, 1, 0]),
  'westage_per_cut': 4.5,
  'total_westage': 2385.0,
  'L': 12}]

#### Opmimize the List

**First Optimization**

In [19]:
def get_sub_cut(item,bar_length_list,bar_number_array):
#     print(item)
    L = item['westage_per_cut']
    item['L'] = L
    total_cut = item['cut']
    bll = bar_length_list
    bna = bar_number_array
    print(item)
    
    results = get_results(L,bna,bll) # Find the Results of All Possible Cuts
    #Now Compare With Total Cuts
    fr = []
    
    for x in results:
        if x['cut']<=total_cut:
            fr.append(x)
            total_cut= total_cut-x['cut']
        else:
            x['cut']= total_cut
            x['total_westage'] = x['cut']*x['westage_per_cut']
            fr.append(x)
            break   
    return fr

In [20]:
final_list = []

In [21]:

for item in distribution_list:
        bar_number_array = bar_number_array-item['combination']*item['cut']
        item['total_westage'] =0
        item['L'] = L
        item['length_arr'] = barLengthList
        final_list.append(item)
        sub_cuts = get_sub_cut(item,barLengthList,bar_number_array)
        for x in sub_cuts:
            final_list.append(x)
            bar_number_array = bar_number_array-x['combination']*x['cut']
    
    
    
    

{'cut': 1, 'combination': array([0, 1, 0, 0, 0, 1]), 'westage_per_cut': 1.5, 'total_westage': 0, 'L': 1.5, 'length_arr': array([1.5, 2. , 4. , 5.5, 7.5, 8.5])}
{'cut': 741, 'combination': array([0, 0, 0, 0, 0, 1]), 'westage_per_cut': 3.5, 'total_westage': 0, 'L': 3.5, 'length_arr': array([1.5, 2. , 4. , 5.5, 7.5, 8.5])}
{'cut': 530, 'combination': array([0, 0, 0, 0, 1, 0]), 'westage_per_cut': 4.5, 'total_westage': 0, 'L': 4.5, 'length_arr': array([1.5, 2. , 4. , 5.5, 7.5, 8.5])}


C:\Users\sohel\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [22]:
for x in sorted_arr:
    cond = bar_number_array ==0
    bal = x[cond]
    if bal.sum() == 0:
        if bar_number_array.sum()==0:
            break
        cond = bar_number_array!=0
        min_arr = bar_number_array/x
        cut = int(np.nanmin(min_arr))
        bar_number_array = bar_number_array - x*cut
        westage = L-(barLengthList*x).sum()
        result = {'cut':cut,'combination':x,'westage_per_cut':westage,'total_westage':cut*westage,'L':L}
        final_list.append(result)
    

C:\Users\sohel\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


In [24]:
for x in final_list:
    x['length_arr']= barLengthList
final_list

[{'cut': 1,
  'combination': array([0, 1, 0, 0, 0, 1]),
  'westage_per_cut': 1.5,
  'total_westage': 0,
  'L': 1.5,
  'length_arr': array([1.5, 2. , 4. , 5.5, 7.5, 8.5])},
 {'cut': 1,
  'combination': array([1, 0, 0, 0, 0, 0]),
  'westage_per_cut': 0.0,
  'total_westage': 0.0,
  'L': 1.5,
  'length_arr': array([1.5, 2. , 4. , 5.5, 7.5, 8.5])},
 {'cut': 741,
  'combination': array([0, 0, 0, 0, 0, 1]),
  'westage_per_cut': 3.5,
  'total_westage': 0,
  'L': 3.5,
  'length_arr': array([1.5, 2. , 4. , 5.5, 7.5, 8.5])},
 {'cut': 98,
  'combination': array([1, 1, 0, 0, 0, 0]),
  'westage_per_cut': 0.0,
  'total_westage': 0.0,
  'L': 3.5,
  'length_arr': array([1.5, 2. , 4. , 5.5, 7.5, 8.5])},
 {'cut': 29,
  'combination': array([0, 1, 0, 0, 0, 0]),
  'westage_per_cut': 1.5,
  'total_westage': 43.5,
  'L': 3.5,
  'length_arr': array([1.5, 2. , 4. , 5.5, 7.5, 8.5])},
 {'cut': 530,
  'combination': array([0, 0, 0, 0, 1, 0]),
  'westage_per_cut': 4.5,
  'total_westage': 0,
  'L': 4.5,
  'length_a

In [25]:
ttt =0
for x in final_list:
    ttt=ttt+x['total_westage']
ttt

645.0